In [13]:
%load_ext autoreload
%autoreload 2

from flask import Flask, request, jsonify
from flask import Blueprint, request, jsonify
from cocomo import cocomo

import math

from io import BytesIO
import os
import base64
import numpy as np
from flask import Flask, make_response, jsonify
from flask import request
from flask_cors import CORS
import json

from FPA import get_FPA_result
from budget import predict_project_viability,get_history_budget

from risk import *
from resource import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def analyze(data):

    # cocomo
    cocomo_input=data['cocomo']  
    # print(cocomo_input)
    project_class = cocomo_input['project_class'].upper()  # 统一转为大写，兼容小写输入
    eaf = float(cocomo_input['eaf'])  # 转换为浮点数，默认值 1
    
    cocomo_output=cocomo(cocomo_input['sloc'],project_class,eaf)
    cocomo_output = {
        **cocomo_output,
        "input_parameters": {
            "sloc": cocomo_input['sloc'],
            "project_class": project_class,  # 保留原始输入格式
            "eaf": eaf
        }
    }
    
    # 加载历史数据
    historical = get_history_budget()['historical']
    # 生成financial预测
    prediction = predict_project_viability(
        cocomo_output,
        historical_data=historical,
        discount_rate=data['discount_rate']
    )
    prediction['discount_rate']=data['discount_rate']
    print(prediction)

    # risk
    risk_analysis = calculate_risk_analysis(cocomo_output)
    print(risk_analysis)
    risk_analysis.update({
        "monteCarloResults": monte_carlo_simulation(prediction['cashFlowAnalysis']),
        "sensitivityAnalysis": perform_sensitivity_analysis(prediction)
    })


    # resource
    # 资源分配与优化（新增部分）
    data['resource_allocation_inputs']=data['resource_allocation_inputs']

    resource_inputs = data['resource_allocation_inputs']
    resource_result = calculate_resource_allocation(
        resource_inputs['tasks'],
        resource_inputs['resource_capacity']
    )
    
    # 填充COCOMO成本到资源结果
    resource_result['totalCost'] = cocomo_output['cost_USD']
    resource_result['optimization']['current']['cost'] = cocomo_output['cost_USD']
    resource_result['optimization']['optimized']['cost'] = cocomo_output['cost_USD'] * 0.95
    resource_result['optimization']['savings']['cost'] = cocomo_output['cost_USD'] * 0.05


    result={
        "cocomo":cocomo_output,
        # "fpa":fpa_output,
        # "budget_metrics": "not available",
        "risk_analysis":risk_analysis,
        "resource_schedule":resource_result,
        "financial":prediction
    }

    
    return result
    
    

In [15]:
app = Flask(__name__)
CORS(app)

@app.route('/analyze', methods=['POST'])
def all_analyze():
    try:
        data = request.get_json()
        
        result = analyze(data)
        
        # 格式化结果
        return jsonify(result), 200

    except Exception as e:
            return jsonify({
                "error": f"Calculation failed: {str(e)}",
                "details": "Check input parameters and ensure they are valid numbers."
            }), 500


@app.route('/history', methods=['GET'])
def get_history_data():
    try:
        
        result = get_history_budget()
        
        # 格式化结果
        return jsonify(result), 200

    except Exception as e:
            return jsonify({
                "error": f"Calculation failed: {str(e)}",
                "details": "Check input parameters and ensure they are valid numbers."
            }), 500


if __name__ == '__main__':
    app.run(
        host='localhost',
        # host='59.65.191.49',
        port=9090,
        debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:9090
Press CTRL+C to quit
127.0.0.1 - - [11/Jun/2025 16:21:06] "POST /analyze HTTP/1.1" 200 -


here 0.1
{'roi': np.float64(-142.44), 'npv': np.float64(-492944.63), 'irr': -22.75, 'payback_period': 'N/A', 'cashFlowAnalysis': [{'period': 'Period 0', 'cashFlow': -938211.35}, {'period': 'Period 1', 'cashFlow': np.float64(165000.0)}, {'period': 'Period 2', 'cashFlow': np.float64(180000.0)}, {'period': 'Period 3', 'cashFlow': np.float64(195000.0)}], 'budgetTracking': {'planned': [938211.35, 0, 0, 0], 'actual': [-938211.35, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)], 'variance': [-1876422.7, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)]}, 'discount_rate': 0.1}
{'totalRiskScore': 0.86, 'riskMatrix': [{'name': 'Technical Complexity Risk', 'probability': 0.36, 'impact': 0.8, 'score': 0.29}, {'name': 'Human Resource Risk', 'probability': 0.24, 'impact': 0.6, 'score': 0.14}, {'name': 'Budget Overrun Risk', 'probability': 0.48, 'impact': 0.9, 'score': 0.43}]}
[{'id': 'T1', 'name': '需求分析', 'duration': 5, 'dependencies': []}, {'id': 'T2', 'name': '系统设

127.0.0.1 - - [11/Jun/2025 16:21:12] "POST /analyze HTTP/1.1" 200 -


here 0.1
{'roi': np.float64(-109.33), 'npv': np.float64(-150326.49), 'irr': -4.62, 'payback_period': 'N/A', 'cashFlowAnalysis': [{'period': 'Period 0', 'cashFlow': -595593.21}, {'period': 'Period 1', 'cashFlow': np.float64(165000.0)}, {'period': 'Period 2', 'cashFlow': np.float64(180000.0)}, {'period': 'Period 3', 'cashFlow': np.float64(195000.0)}], 'budgetTracking': {'planned': [595593.21, 0, 0, 0], 'actual': [-595593.21, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)], 'variance': [-1191186.42, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)]}, 'discount_rate': 0.1}
{'totalRiskScore': 0.86, 'riskMatrix': [{'name': 'Technical Complexity Risk', 'probability': 0.36, 'impact': 0.8, 'score': 0.29}, {'name': 'Human Resource Risk', 'probability': 0.24, 'impact': 0.6, 'score': 0.14}, {'name': 'Budget Overrun Risk', 'probability': 0.48, 'impact': 0.9, 'score': 0.43}]}
[{'id': 'T1', 'name': '需求分析', 'duration': 5, 'dependencies': []}, {'id': 'T2', 'name': '系统设

127.0.0.1 - - [11/Jun/2025 16:23:00] "POST /analyze HTTP/1.1" 200 -


here 0.01
{'roi': np.float64(-109.33), 'npv': np.float64(-66508.51), 'irr': -4.62, 'payback_period': 'N/A', 'cashFlowAnalysis': [{'period': 'Period 0', 'cashFlow': -595593.21}, {'period': 'Period 1', 'cashFlow': np.float64(165000.0)}, {'period': 'Period 2', 'cashFlow': np.float64(180000.0)}, {'period': 'Period 3', 'cashFlow': np.float64(195000.0)}], 'budgetTracking': {'planned': [595593.21, 0, 0, 0], 'actual': [-595593.21, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)], 'variance': [-1191186.42, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)]}, 'discount_rate': 0.01}
{'totalRiskScore': 0.86, 'riskMatrix': [{'name': 'Technical Complexity Risk', 'probability': 0.36, 'impact': 0.8, 'score': 0.29}, {'name': 'Human Resource Risk', 'probability': 0.24, 'impact': 0.6, 'score': 0.14}, {'name': 'Budget Overrun Risk', 'probability': 0.48, 'impact': 0.9, 'score': 0.43}]}
[{'id': 'T1', 'name': '需求分析', 'duration': 5, 'dependencies': []}, {'id': 'T2', 'name': '系统

127.0.0.1 - - [11/Jun/2025 16:23:06] "POST /analyze HTTP/1.1" 200 -


here 0.01
{'roi': np.float64(110.33), 'npv': np.float64(355074.81), 'irr': 84.08, 'payback_period': np.float64(2.95), 'cashFlowAnalysis': [{'period': 'Period 0', 'cashFlow': -174009.89}, {'period': 'Period 1', 'cashFlow': np.float64(165000.0)}, {'period': 'Period 2', 'cashFlow': np.float64(180000.0)}, {'period': 'Period 3', 'cashFlow': np.float64(195000.0)}], 'budgetTracking': {'planned': [174009.89, 0, 0, 0], 'actual': [-174009.89, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)], 'variance': [-348019.78, np.float64(165000.0), np.float64(180000.0), np.float64(195000.0)]}, 'discount_rate': 0.01}
{'totalRiskScore': 0.86, 'riskMatrix': [{'name': 'Technical Complexity Risk', 'probability': 0.36, 'impact': 0.8, 'score': 0.29}, {'name': 'Human Resource Risk', 'probability': 0.24, 'impact': 0.6, 'score': 0.14}, {'name': 'Budget Overrun Risk', 'probability': 0.48, 'impact': 0.9, 'score': 0.43}]}
[{'id': 'T1', 'name': '需求分析', 'duration': 5, 'dependencies': []}, {'id': 'T2', 'n